In [2]:
pip install fastapi


     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/149.4 kB ? eta -:--:--
     -- ------------------------------------- 10.2/149.4 kB ? eta -:--:--
     ------- ----------------------------- 30.7/149.4 kB 217.9 kB/s eta 0:00:01
     ------- ----------------------------- 30.7/149.4 kB 217.9 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/149.4 kB 150.6 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/149.4 kB 150.6 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/149.4 kB 150.6 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/149.4 kB 150.6 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/149.4 kB 150.6 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/149.4 kB 150.6 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/149.4 kB 150.6 kB

In [4]:
pip install deepface


     ---------------------------------------- 0.0/88.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/88.4 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/88.4 kB ? eta -:--:--
     ------------- ------------------------ 30.7/88.4 kB 330.3 kB/s eta 0:00:01
     ------------- ------------------------ 30.7/88.4 kB 330.3 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 219.4 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 219.4 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 219.4 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 219.4 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 219.4 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 219.4 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.4 kB 219.4 kB/s eta 0:00:01
     -------------------------- ----------- 61.4/88.4 kB 109.4 

In [7]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 186.2 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.7 MB 65.5 kB/s eta 0:00:26
    -------------------

In [9]:
pip install python-multipart


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install uvicorn


  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/63.7 kB ? eta -:--:--
   ------ --------------------------------- 10.2/63.7 kB ? eta -:--:--
   ------ --------------------------------- 10.2/63.7 kB ? eta -:--:--
   ------ --------------------------------- 10.2/63.7 kB ? eta -:--:--
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   ------------------- -------------------- 30.7/63.7 kB 146.3 kB/s eta 0:00:01
   -------------

In [13]:
from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
from PIL import Image
import numpy as np
import torch
import cv2
from ultralytics import YOLO
from io import BytesIO
from fastapi.middleware.cors import CORSMiddleware
from ultralytics.engine.results import Results
from deepface import DeepFace
import tensorflow as tf
import h5py

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], 
    allow_credentials=True,
    allow_methods=["*"], 
    allow_headers=["*"], 
)
# Load model Fashion
fashion_model = YOLO("C:\\Users\\camnh\\Downloads\\E-Fas\\best.pt")

class GenderResponse(BaseModel):
    label: str
    confidence: float


class FashionResponse(BaseModel):
    cropped_image: bytes
    label: str
    confidence: float


def load_image_into_numpy_array(data):
    return np.array(Image.open(BytesIO(data)))


@app.post("/detect-gender/", response_model=GenderResponse)
async def detect_gender(file: UploadFile = File(...)):
    img = load_image_into_numpy_array(await file.read())
    result = DeepFace.analyze(img, actions=['gender'])
    # Run gender model
    if isinstance(result, list):
        result = result[0]

    gender_confidence = result['gender']
    if gender_confidence['Woman'] > gender_confidence['Man']:
        highest_gender = 'Woman'
        highest_confidence = gender_confidence['Woman']
        return GenderResponse(label=highest_gender, confidence=highest_confidence)
    else:
        highest_gender = 'Man'
        highest_confidence = gender_confidence['Man']
        return GenderResponse(label=highest_gender, confidence=highest_confidence)


@app.post("/detect-fashion/", response_model=FashionResponse)
async def detect_fashion(file: UploadFile = File(...)):
    img = load_image_into_numpy_array(await file.read())
    class_list=['bag', 'dress', 'hat', 'jacket', 'pants', 'shirt', 'shoe', 'shorts', 'skirt', 'sunglass']
    # Run fashion model
    results = fashion_model(img)
    result = results[0]
    data=result.boxes
    # Output
    labels = data.cls.tolist()
    detections = data.xyxy.numpy()

    # Check if any detections were made
    cropped_images = []  # Danh sách để chứa tất cả các ảnh đã cắt

    # Duyệt qua từng bounding box
    for i, detection in enumerate(detections):
        label_idx = int(labels[i])
        label = class_list[label_idx]  # Nhãn tương ứng với bounding box
        
        # Tọa độ bounding box
        xmin, ymin, xmax, ymax = map(int, detection[:4])
        confidence = float(data.conf[i])  # Độ tin cậy của nhãn

        # Cắt ảnh theo bounding box
        cropped_img = img[ymin:ymax, xmin:xmax]
        
        # Encode the cropped image to bytes
        _, encoded_img = cv2.imencode('.jpg', cropped_img)
        cropped_image_bytes = encoded_img.tobytes()
    
    else:
        cropped_image_bytes = None
        label = "Unknown"
        confidence = 0.0

    return FashionResponse(
        cropped_image=cropped_image_bytes,
        label=label,
        confidence=confidence
    )


if __name__ == "__main__":
    import uvicorn
    uvicorn.run("YOLO_Fashion_API:app", host="127.0.0.1", port=8000, reload=True)

INFO:     Will watch for changes in these directories: ['d:\\API_FASHION']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [15512] using StatReload
INFO:     Stopping reloader process [15512]
